In [1]:
from shapely.geometry import LineString, Point, MultiPoint, Polygon

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import geopandas as gpd
import numpy as np
from datetime import datetime
from shapely.geometry import LineString, Point, MultiPoint, Polygon
from IPython.display import display, clear_output

df = pd.read_csv('data/train.csv')

starts = gpd.GeoDataFrame(df[['id']], geometry=gpd.points_from_xy(df.pickup_longitude, df.pickup_latitude), crs=4326)
stops = gpd.GeoDataFrame(df[['id']], geometry=gpd.points_from_xy(df.dropoff_longitude, df.dropoff_latitude), crs=4326)

path_to_data = gpd.datasets.get_path("nybb")
gdf = gpd.read_file(path_to_data)
manh = gdf.loc[gdf.BoroName=='Manhattan'].to_crs(4326)

roads = pd.read_pickle('roadsfornx.pkl')

sind = gpd.tools.sjoin(starts, manh, op='within').index.tolist()
eind = gpd.tools.sjoin(stops, manh, op='within').index.tolist()
df = df.loc[list(set(sind).intersection(set(eind)))]
starts = gpd.GeoDataFrame(df[['id']], geometry=gpd.points_from_xy(df.pickup_longitude, df.pickup_latitude), crs=4326)
stops = gpd.GeoDataFrame(df[['id']], geometry=gpd.points_from_xy(df.dropoff_longitude, df.dropoff_latitude), crs=4326)

reps = pd.read_pickle('reps.pkl')

In [17]:
from routing import route_new
from misc import viz2

In [19]:
for i in range(100):

    rt = route_new(starts.iloc[i].geometry, stops.iloc[i].geometry, roads, reps)

    # viz2(rt, starts.iloc[i:i+1], stops.iloc[i:i+1])
    # input()
    # clear_output()


  0%|          | 0/13630 [00:00<?, ?it/s]

  0%|          | 0/13630 [00:00<?, ?it/s]

  0%|          | 0/13630 [00:00<?, ?it/s]

  0%|          | 0/13630 [00:00<?, ?it/s]

  0%|          | 0/13630 [00:00<?, ?it/s]

...

  0%|          | 0/13630 [00:00<?, ?it/s]

  0%|          | 0/13630 [00:00<?, ?it/s]

  0%|          | 0/13630 [00:00<?, ?it/s]

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniforge/base/envs/gis/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3397, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/jk/dh__d901235bn105dxr5db7c0000gn/T/ipykernel_60143/3160256328.py", line 3, in <cell line: 1>
    rt = route_new(starts.iloc[i].geometry, stops.iloc[i].geometry, roads, reps)
  File "/Users/georgetheodoropoulos/Code/URF/routing.py", line 17, in route_new
    G.add_edge(a, b, weight=c)
  File "/opt/homebrew/Caskroom/miniforge/base/envs/gis/lib/python3.9/site-packages/networkx/classes/digraph.py", line 636, in add_edge
    self._pred[v][u] = datadict
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniforge/base/envs/gis/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 1992, in showtraceback
    stb = self.InteractiveT

In [14]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
